<a href="https://colab.research.google.com/github/amirabbasgashtil/Linear-Programming/blob/main/simplex_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex algorithm for optimizing a characteristic function

# first we have to input characteristic function and constraints

In [ ]:
z = []
z =[int(item) for item in input('Enter coeficiant of characteristic function : ').split()]
print(z)

In [ ]:
m = input('Enter the type of problem: (min or max) : ')

In [ ]:
number_of_constraints = int(input('Enter the number of constraints: '))

In [ ]:
A = [[int(input('Enter coeficiant of x{} in constraint{} : '.format(i+1, c+1))) for i in range(len(z))] for c in range(number_of_constraints)]

In [ ]:
Draws = [input('Enter the Draw of constraint{} : '.format(i+1)) for i in range(number_of_constraints)]

In [ ]:
RHS = [int(input('Enter Right Hand Side in constraint{} : '.format(i+1))) for i in range(number_of_constraints)]

In [ ]:
sign_of_variables = [input('Enter the sign of x{} : (>= / <= / free) '.format(i+1)) for i in range(len(z))]

# Now lets convert form of problem to its standard form

1. convert all of constraints to =
2. for all x in characteristic function =>  x>=0

1)

In [ ]:
for i in range(len(Draws)):

  if Draws[i] == ">=":
    # we have to add a (-1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[j].append(-1)
      else:
        A[j].append(0)

  elif Draws[i] == "<=":
    # we have to add a (1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[i].append(1)
      else:
        A[i].append(0)

print(A)
print(z)

2)

In [ ]:
for i in range(len(sign_of_variables)):
  if sign_of_variables[i] == 'free':
    z.insert(i+1, -1 * z[i])
    for j in range(number_of_constraints):
        A[j].insert(i+1, -1 * A[j][i])
  elif sign_of_variables[i] == '<=':
    z[i] = -1 * z[i]
    for j in range(number_of_constraints):
      A[j][i] = -1 * A[j][i]

print(z)
print(A)

In [ ]:
'''
    now we have an standard optimization problem
    lets print the standard form and then solve it by simplex algorithm
'''
print(m, end="  ")
print('z = ', end="")
for i in range(len(z)):
  print(str(z[i]) + 'x.{}'.format(i+1), end=" ")
print()
print('subject to : ')
for i in range(number_of_constraints):
  for j in range(len(z)):
    print(str(A[i][j]) + '.x{}'.format(j+1), end=" ")
  print('=', end=" ")
  print(RHS[i])

for i in range(len(z)):
  print('x.{}'.format(i+1), end=" ")
print('>= 0')

# simplex method for optimization

first we have to know our problem got the answer with regular simplex method or needs to another simplex method

In [ ]:
def reg_simplex(A_simplex, z, number_of_constraints):
  counter = 0
  base_counter = 0
  status = False
  for i in range(number_of_constraints):
    for j in range(len(z)):
      if A_simplex[i][j] == 1:
        for k in range(number_of_constraints):
          if k != i:
            if A_simplex[k][j] != 0:
              status = False
            else:
              counter += 1
        if counter == number_of_constraints - 1:
          base_counter += 1
  if base_counter == number_of_constraints:
    return True
  else:
    return False

In [ ]:
def first_x_b(x_b,number_of_constraints, z, A_simplex):
  counter = 0
  for i in range(number_of_constraints):
    for j in range(len(z)):
      if A_simplex[i][j] == 1:
        for k in range(number_of_constraints):
          if k != i and A_simplex[k][j] == 0:
            counter += 1
        if counter == number_of_constraints - 1:
          x_b.append(j)
        counter = 0
  return x_b

In [ ]:
def first_c_b(c_b, x_b, z):
  for i in range(len(x_b)):
    c_b.append(z[x_b[i]])
  return

In [ ]:
def first_c_bar(c_bar, c_b, A_simplex):
  for i in range(len(c_bar)):
    for j in range(len(c_b)):
      c_bar[i] -= A_simplex[j][i] * c_b[j]
  return

In [ ]:
def first_z(c_b, sim_RHS):
  z_val = 0
  for i in range(len(c_b)):
    z_val += c_b[i] * sim_RHS[i]
  return z_val

In [ ]:
def check_for_improve(c_bar, x_b):
  if m == 'min':
    for i in range(len(c_bar)):
      if i != x_b and c_bar[i] < 0:
        return True
    return False

  else:
    for i in range(len(c_bar)):
      if i != x_b and c_bar[i] > 0:
        return True
    return False

In [ ]:
def minimum_test(add_to_cb, A_simplex, sim_RHS):
  candidates = []
  for i in range(len(sim_RHS)):
    if A_simplex[i][add_to_cb] <= 0:
      candidates.append(1000)
    else:
      candidates.append(round((sim_RHS[i]/A_simplex[i][add_to_cb]), 2))
  temp = min(candidates)
  for i in range(len(candidates)):
    if candidates[i] == temp:
      candidates.clear()
      return i

In [ ]:
def row_reduction(A_simplex, c_bar, i, j, sim_RHS):
  divide = A_simplex[j][i]

  for k in range(len(c_bar)):
    A_simplex[j][k] = round(A_simplex[j][k] / divide, 2)
  sim_RHS[j] = round(sim_RHS[j] / divide, 2)

  for k in range(len(sim_RHS)):
    x = A_simplex[k][i]
    if k != j:
      for t in range(len(c_bar)):
          A_simplex[k][t] = (A_simplex[k][t] - (x * A_simplex[j][t]))
      sim_RHS[k] = sim_RHS[k] - (x * sim_RHS[j])

  y = c_bar[i]
  for k in range(len(c_bar)):
    c_bar[k] = c_bar[k] - (y * A_simplex[j][i])
  return

In [ ]:
def new_z(z_value, c_b, sim_RHS):
  z_value = 0
  for i in range(len(c_b)):
    z_value += c_b[i] * sim_RHS[i]
  return

In [ ]:
def tableau(z, c_b, x_b, A_simplex, sim_RHS, c_bar, z_value):
  print('********************')
  print('c_b',end="|")
  print('x_b', end="|")
  print(z, end="|")
  print('RHS')
  for i in range(len(sim_RHS)):
    print(c_b[i],end="|")
    print(x_b[i],end="|")
    for j in range(len(z)):
      print(A_simplex[i][j], end=" ")
    print(sim_RHS[i])
  print('C_bar : ', end="|")
  for i in range(len(z)):
    print(c_bar[i], end=" ")
  print(z_value)
  print('********************')
  return

In [ ]:
def improve(c_b, x_b, z, c_bar, A_simplex, sim_RHS, z_value):
  if m == 'min':
    add_to_cb = min(c_bar)
    for i in range(len(c_bar)):
      if c_bar[i] == add_to_cb:
        add_to_cb = i
        break
  else:
    add_to_cb = max(c_bar)
    for i in range(len(c_bar)):
      if c_bar[i] == add_to_cb:
        add_to_cb = i
        break

    j = minimum_test(add_to_cb, A_simplex, sim_RHS)
    x_b[j] = i
    c_b[j] = z[i]
    row_reduction(A_simplex, c_bar, i, j, sim_RHS)
    new_z(z_value, c_b, sim_RHS)

    tableau(z, c_b, x_b, A_simplex, sim_RHS, c_bar, z_value)
  return

In [ ]:
def simplex(A_simplex,number_of_constraints, z, RHS):
  # first values:
  c_b = []
  x_b = []
  sim_RHS = RHS
  c_bar = []

  for i in range(len(z)):
    c_bar.append(z[i])

  first_x_b(x_b, number_of_constraints, z, A_simplex)
  first_c_b(c_b, x_b, z)
  first_c_bar(c_bar, c_b, A_simplex)

  z_value = first_z(c_b, sim_RHS)



  tableau(z, c_b, x_b, A_simplex, sim_RHS, c_bar, z_value)
  # try to improve
  while(check_for_improve(c_bar, x_b)):
    improve(c_b, x_b, z, c_bar, A_simplex, sim_RHS, z_value)
  return

In [ ]:
def bigM_simplex():
  return

In [ ]:
A_simplex = A
if reg_simplex(A_simplex, z, number_of_constraints):
  simplex(A_simplex, number_of_constraints, z, RHS)
else:
  bigM_simplex()